#### Файл сервера xmlrpc_stats_server_p3.ipynb на Python 3 

In [ ]:
from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler
import datetime
import csv
import os
import re
from datetime import datetime, timedelta

class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

# Включаем поддержку None
server = SimpleXMLRPCServer(("127.0.0.1", 8018), requestHandler=RequestHandler, allow_none=True)

MAX_COUNT_LINES = 100

# Изменить MAX_COUNT_LINES
def change_limit(limit):
    global MAX_COUNT_LINES
    if isinstance(limit, int) and limit > 0:
        MAX_COUNT_LINES = limit
        return True
    return False
server.register_function(change_limit, 'change_limit')

# Тест
def ping():
    return True
server.register_function(ping, 'ping')

# Время сервера
def now():
    return datetime.now()  # Исправлено: убрано лишнее .datetime
server.register_function(now, 'now')

# Подсчет строк в файле
def count_lines(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return sum(1 for line in f)
    except FileNotFoundError:
        return 0

# Добавление строки в лог
def add_log(sname):
    log_file = 'logs/logs.csv'
    os.makedirs('logs', exist_ok=True)

    # Получаем текущую временную метку один раз
    current_time = datetime.now()
    timestamp = current_time.strftime('%Y%m%d_%H%M%S')
    formatted_time = current_time.strftime('%Y-%m-%d %H:%M:%S')

    # Проверяем количество строк
    line_count = count_lines(log_file)
    if line_count >= MAX_COUNT_LINES:
        # Формируем имя архивного файла
        archive_file = f'logs/logs_{timestamp}.csv'
        # Переименовываем текущий файл
        if os.path.exists(log_file):
            os.rename(log_file, archive_file)
        # Создаем новый пустой logs.csv с заголовками
        with open(log_file, 'w', encoding='utf-8', newline='') as f:
            writer = csv.writer(f, delimiter=';')
            writer.writerow(['Event', 'Timestamp'])

    # Добавляем новую запись
    with open(log_file, 'a', encoding='utf-8', newline='') as f:
        writer = csv.writer(f, delimiter=';')
        writer.writerow([sname, formatted_time])
    return True
server.register_function(add_log, 'add_log')

# Получение содержимого журнала с фильтрацией
def get_log(event_filter=False, start_time=False, end_time=False):
    log_dir = 'logs'
    log_file = f'{log_dir}/logs.csv'
    filtered_entries = []

    try:
        # Список логов
        log_files = [f for f in os.listdir(log_dir) if f.startswith('logs') and f.endswith('.csv')]
    except FileNotFoundError:
        log_files = []

    if os.path.exists(log_file) and 'logs.csv' not in log_files:
        log_files.append('logs.csv')

    # Преобразание start_time и end_time в datetime
    start_dt = None
    end_dt = None
    if start_time:
        try:
            start_dt = datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')
        except ValueError:
            print(f'Ошибка: Неверный формат start_time: {start_time}. Ожидается YYYY-MM-DD HH:MM:SS')
            return []
    if end_time:
        try:
            end_dt = datetime.strptime(end_time, '%Y-%m-%d %H:%M:%S')
        except ValueError:
            print(f'Ошибка: Неверный формат end_time: {end_time}. Ожидается YYYY-MM-DD HH:MM:SS')
            return []

    # Фильтр файлов
    relevant_files = []
    for f in log_files:
        if f == 'logs.csv':
            relevant_files.append(f)
        else:
            match = re.match(r'logs_(\d{8})_(\d{6})\.csv', f)
            if match:
                file_time = datetime.strptime(f'{match.group(1)} {match.group(2)}', '%Y%m%d %H%M%S')
                if not start_dt or not end_dt or (file_time <= end_dt + timedelta(hours=1)):
                    relevant_files.append(f)

    for f in relevant_files:
        file_path = os.path.join(log_dir, f)
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                reader = csv.reader(file, delimiter=';')
                next(reader, None)
                for row in reader:
                    if len(row) != 2:
                        continue
                    event, timestamp = row
                    if timestamp == 'Timestamp':
                        continue

                    # Фильтр по событию
                    if event_filter and event_filter.lower() not in event.lower():
                        continue

                    # Фильтр по времени
                    try:
                        row_time = datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')
                        if start_dt and row_time < start_dt:
                            continue
                        if end_dt and row_time > end_dt:
                            continue
                    except ValueError:
                        continue

                    filtered_entries.append([event, timestamp])
        except FileNotFoundError:
            continue

    return filtered_entries
server.register_function(get_log, 'get_log')

print("Listening on port 8018...")
server.serve_forever()

Listening on port 8018...


127.0.0.1 - - [22/Sep/2025 19:59:03] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2025 19:59:03] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2025 19:59:11] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2025 19:59:11] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2025 19:59:16] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2025 19:59:57] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2025 20:00:02] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2025 20:00:05] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2025 20:00:09] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2025 20:00:14] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2025 20:00:29] "POST /RPC2 HTTP/1.1" 200 -


Ошибка: Неверный формат start_time: 2025-09-20 12:10. Ожидается YYYY-MM-DD HH:MM:SS


127.0.0.1 - - [22/Sep/2025 20:00:31] "POST /RPC2 HTTP/1.1" 200 -


Ошибка: Неверный формат start_time: 2025-09-20 12:10. Ожидается YYYY-MM-DD HH:MM:SS


...

127.0.0.1 - - [22/Sep/2025 20:03:08] "POST /RPC2 HTTP/1.1" 200 -


Ошибка: Неверный формат start_time: 2025-09-20 12:10. Ожидается YYYY-MM-DD HH:MM:SS


127.0.0.1 - - [22/Sep/2025 20:03:34] "POST /RPC2 HTTP/1.1" 200 -


Ошибка: Неверный формат start_time: 2025-09-20 12:10. Ожидается YYYY-MM-DD HH:MM:SS


127.0.0.1 - - [22/Sep/2025 20:03:37] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2025 20:03:40] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2025 20:03:54] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2025 20:05:54] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2025 20:05:57] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2025 20:07:02] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2025 20:07:11] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2025 20:07:34] "POST /RPC2 HTTP/1.1" 200 -
